# Session 1.4: BakeryAI - Conversation Memory & Personalization

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1F5vDIiTiNAA09oBKOvfWy5Oqo45mZYRS?usp=sharing)

## 🎯 Final Session Goal

Complete BakeryAI with **conversational memory and personalization**!

### What We'll Build:

✅ Multi-turn conversations that remember context  
✅ Customer preference tracking  
✅ Session-based order building  
✅ Personalized product recommendations  
✅ Complete BakeryAI v1.0  

### Why This Matters:

Natural conversation is key to great customer service:
- **Context Awareness**: "Add that to my order" → knows what "that" refers to
- **Personalization**: Remembers dietary preferences, past orders
- **Efficiency**: No need to repeat information
- **Experience**: Feels like talking to a real person

### 🚀 BakeryAI Progress: 60% → 100%
```
[████████████████████] 100%
```

## 🎉 Let's Complete BakeryAI!

In [1]:
!pip install -q langchain langchain-openai langchain-community

In [2]:
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory
)
from langchain.chains import ConversationChain
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [3]:
import os
from google.colab import userdata

# Set OpenAI API key from Google Colab's user environment or default
def set_openai_api_key(default_key: str = "YOUR_API_KEY") -> None:
    """Set the OpenAI API key from Google Colab's user environment or use a default value."""
    #if not (userdata.get("OPENAI_API_KEY") or "OPENAI_API_KEY" in os.environ):
    try:
      os.environ["OPENAI_API_KEY"] = userdata.get("MDX_OPENAI_API_KEY")
    except:
      os.environ["OPENAI_API_KEY"] = default_key

set_openai_api_key()

In [4]:
!git clone https://github.com/IvanReznikov/mdx-langchain-conclave

fatal: destination path 'mdx-langchain-conclave' already exists and is not an empty directory.


In [5]:
llm = ChatOpenAI(model="gpt-5-nano")
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# Load data
try:
    cakes_df = pd.read_csv('/content/mdx-langchain-conclave/data/cake_descriptions.csv', encoding='cp1252')
    customers_df = pd.read_csv('/content/mdx-langchain-conclave/data/customers.csv', encoding='cp1252')
except FileNotFoundError:
    cakes_df = pd.DataFrame({
        'Name': ['Chocolate Truffle', 'Vanilla Bean', 'Red Velvet', 'Lemon Drizzle'],
        'Description': ['Rich chocolate', 'Classic vanilla', 'Velvety red', 'Citrus burst'],
        'Energy_kcal': [450, 380, 420, 320],
        'Category': ['Chocolate', 'Vanilla', 'Specialty', 'Fruit']
    })

print("✅ BakeryAI Memory System Ready!")

✅ BakeryAI Memory System Ready!


## 1. Basic Conversation Memory for BakeryAI

Let's start with simple memory to remember customer interactions.

In [6]:
# Create BakeryAI with memory
def create_bakery_context():
    return f"""
    Available Products:
    {cakes_df[['Name', 'Description', 'Energy_kcal']].to_string()}
    """

bakery_system_prompt = f"""
You are BakeryAI, a friendly bakery assistant.

{create_bakery_context()}

Your role:
- Help customers find perfect products
- Remember their preferences throughout the conversation
- Build orders step-by-step
- Be warm, helpful, and professional
"""

# Initialize memory
memory = ConversationBufferMemory(return_messages=True)

# Create conversation chain
bakery_conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

# Override the default prompt
bakery_conversation.prompt.template = bakery_system_prompt + "\n\nConversation:\n{history}\nCustomer: {input}\nBakeryAI:"

print("🍰 BakeryAI is ready to chat!")

🍰 BakeryAI is ready to chat!


/tmp/ipython-input-1969987168.py:21: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True)
/tmp/ipython-input-1969987168.py:24: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  bakery_conversation = ConversationChain(


In [7]:
# Simulate a customer conversation
conversation_flow = [
    "Hi! I'm looking for a cake for my daughter's birthday",
    "She loves chocolate",
    "How many calories does the chocolate one have?",
    "Perfect! Can I order it for delivery tomorrow?"
]

print("🗣️  Customer Conversation Simulation\n")
print("=" * 70)

for message in conversation_flow:
    print(f"\n👤 Customer: {message}")
    response = bakery_conversation.predict(input=message)
    print(f"🍰 BakeryAI: {response}")
    print("-" * 70)

🗣️  Customer Conversation Simulation


👤 Customer: Hi! I'm looking for a cake for my daughter's birthday


> Entering new ConversationChain chain...
Prompt after formatting:

You are BakeryAI, a friendly bakery assistant.


    Available Products:
                                      Name                                                             Description  Energy_kcal
0              Torta della Nonna Amore   A nostalgic Tuscan-inspired cake with creamy ricotta and lemon warmth          320
1                Festiva della Sicilia  A zesty almond cake with mascarpone frosting and Sicilian citrus flair          360
2     Dubai Midnight Pistachio Fantasy    Lush green cake with exotic floral pistachio notes and white ganache          420
3                Ferrari Redline Fudge        Dense chocolate fudge cake with red mirror glaze and a bold kick          450
4   Goalpost Delight - Soccer Fan Cake              Themed cake with stadium design, perfect for football fans          390
5   

In [8]:
# Check what BakeryAI remembers
print("\n📝 Conversation Memory:")
print(memory.load_memory_variables({}))


📝 Conversation Memory:
{'history': [HumanMessage(content="Hi! I'm looking for a cake for my daughter's birthday", additional_kwargs={}, response_metadata={}), AIMessage(content='Happy birthday to your daughter! I’d love to help you find the perfect cake. Do you have any ideas or a theme in mind (colors, characters, sports, etc.)?\n\nHere are a few birthday-friendly options to consider:\n- Rainbow Cake (Colorful layers, great for celebrations) — 390 kcal\n- Strawberry Shortcake (Light, fruity, kid-friendly) — 360 kcal\n- ChocoCaramel Birthday Surprise (Rich chocolate with hidden caramel) — 470 kcal\n- Velvet Dream Cloud (Fluffy and elegant, still a treat) — 350 kcal\n\nIf you’re aiming for a specific theme, I can suggest decorations and inscriptions to match.\n\nA few quick questions to tailor the order:\n1) How many servings or what cake size do you need?\n2) Any dietary needs or restrictions (nut-free, dairy-free, gluten-free, vegan)?\n3) When and where would you like pickup or deliv

## 2. Session-Based Order Building

Track order details throughout the conversation.

In [9]:
class BakeryOrderSession:
    """Manage order state throughout conversation"""

    def __init__(self, customer_name=None):
        self.customer_name = customer_name
        self.cart = []
        self.preferences = {}
        self.delivery_info = {}
        self.memory = ConversationBufferMemory(return_messages=True)

    def add_to_cart(self, product, quantity=1, customization=None):
        """Add item to cart"""
        self.cart.append({
            'product': product,
            'quantity': quantity,
            'customization': customization,
            'added_at': datetime.now()
        })
        return f"Added {quantity}x {product} to cart"

    def update_preferences(self, preference_type, value):
        """Track customer preferences"""
        self.preferences[preference_type] = value
        return f"Noted: {preference_type} = {value}"

    def set_delivery_info(self, **kwargs):
        """Set delivery details"""
        self.delivery_info.update(kwargs)
        return "Delivery info updated"

    def get_cart_summary(self):
        """Get current cart contents"""
        if not self.cart:
            return "Your cart is empty"

        summary = f"Cart for {self.customer_name or 'Customer'}:\n"
        for item in self.cart:
            summary += f"  - {item['quantity']}x {item['product']}"
            if item['customization']:
                summary += f" ({item['customization']})"
            summary += "\n"
        return summary

    def get_full_context(self):
        """Get complete session context"""
        return {
            'customer': self.customer_name,
            'cart': self.cart,
            'preferences': self.preferences,
            'delivery': self.delivery_info
        }

# Test the order session
session = BakeryOrderSession(customer_name="Sarah")
print(session.add_to_cart("Chocolate Truffle Cake", quantity=1))
print(session.update_preferences("dietary", "no nuts"))
print(session.set_delivery_info(date="tomorrow", time="2 PM"))
print("\n" + session.get_cart_summary())

Added 1x Chocolate Truffle Cake to cart
Noted: dietary = no nuts
Delivery info updated

Cart for Sarah:
  - 1x Chocolate Truffle Cake



## 3. Smart BakeryAI with Order Tracking

In [10]:
class SmartBakeryAI:
    """Complete BakeryAI with memory and order management"""

    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
        self.session = None

    def start_session(self, customer_name=None):
        """Start a new customer session"""
        self.session = BakeryOrderSession(customer_name)
        return f"Welcome{' ' + customer_name if customer_name else ''}! How can I help you today?"

    def chat(self, message):
        """Process customer message with full context"""
        if not self.session:
            self.start_session()

        # Build context-aware prompt
        context = f"""
        Current Session State:
        {self.session.get_cart_summary()}

        Customer Preferences: {self.session.preferences}

        Available Products:
        {cakes_df[['Name', 'Description']].to_string()}

        You are BakeryAI. Use the session state to provide contextual responses.
        If the customer wants to add something, update the cart.
        Remember their preferences throughout the conversation.

        Customer: {message}
        """

        response = self.llm.invoke(context)

        # Save to memory
        self.session.memory.save_context(
            {"input": message},
            {"output": response.content}
        )

        return response.content

    def get_conversation_history(self):
        """Get full conversation"""
        if self.session:
            return self.session.memory.load_memory_variables({})
        return None

# Test SmartBakeryAI
smart_bakery = SmartBakeryAI()
print(smart_bakery.start_session(customer_name="Michael"))
print()

# Simulate conversation
exchanges = [
    "I need something for an anniversary",
    "My wife loves rich, indulgent desserts",
    "The chocolate one sounds perfect! Add it to my order",
    "Can you deliver on Friday evening?"
]

for msg in exchanges:
    print(f"👤 {msg}")
    response = smart_bakery.chat(msg)
    print(f"🍰 {response}\n")

Welcome Michael! How can I help you today?

👤 I need something for an anniversary
🍰 BakeryAI: How about our Trophy Cake – Champions Edition? It's a tall, layered cake shaped like a trophy, perfect for celebrating an anniversary with indulgent flavors. Would you like to add it to your cart?

👤 My wife loves rich, indulgent desserts
🍰 BakeryAI: Based on your wife's preference for rich and indulgent desserts, I would recommend our Ferrari Redline Fudge cake or the ChocoCaramel Birthday Surprise cake. Both have decadent flavors that she might enjoy. Would you like to add any of these to your cart?

👤 The chocolate one sounds perfect! Add it to my order
🍰 BakeryAI: Great choice! I have added the Chocolate Truffle Cake to your order. Is there anything else you would like to add or modify in your order?

👤 Can you deliver on Friday evening?
🍰 BakeryAI: Yes, we can arrange delivery for Friday evening. Would you like to add any specific cakes to your order for that day?



## 4. Multi-Customer Session Management

Handle multiple customers with separate memories.

In [11]:
# Store for all customer sessions
customer_sessions = {}

def get_customer_session(customer_id):
    """Get or create customer session"""
    if customer_id not in customer_sessions:
        customer_sessions[customer_id] = ChatMessageHistory()
    return customer_sessions[customer_id]

# Create LCEL chain with history
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content=bakery_system_prompt),
    MessagesPlaceholder(variable_name="history"),
    HumanMessage(content="{input}")
])

chain = prompt | llm

chain_with_history = RunnableWithMessageHistory(
    chain,
    get_customer_session,
    input_messages_key="input",
    history_messages_key="history"
)

# Test with multiple customers
print("👥 Multi-Customer Test\n")

# Customer 1
config_c1 = {"configurable": {"session_id": "customer_001"}}
r1 = chain_with_history.invoke(
    {"input": "Hi, I'm allergic to nuts. What can I order?"},
    config=config_c1
)
print(f"[Customer 1] {r1.content}\n")

# Customer 2
config_c2 = {"configurable": {"session_id": "customer_002"}}
r2 = chain_with_history.invoke(
    {"input": "I love chocolate! Show me your best chocolate cakes"},
    config=config_c2
)
print(f"[Customer 2] {r2.content}\n")

# Back to Customer 1
r3 = chain_with_history.invoke(
    {"input": "Do the cakes you mentioned have nuts?"},
    config=config_c1
)
print(f"[Customer 1 - Follow-up] {r3.content}\n")

print("✅ BakeryAI remembered Customer 1's allergy!")

👥 Multi-Customer Test

[Customer 1] Hi there! I’m BakeryAI, here to help you find the perfect cake and build your order step by step. 

To get started, a quick snapshot of our range (just to help you decide):
- Chocolate favorites: Ferrari Redline Fudge, Chocolate Truffle Cake, Black Forest Cake
- Fruity/bright: Lemon Drizzle Cake, Strawberry Shortcake, Rainbow Cake, Molten Mango Lava Cake
- Creamy/coffee: Tiramisu Cake, Cheesecake, Velvet Dream Cloud
- Nutty/caramel: Hazelnut Toffee Crown, ChocoCaramel Birthday Surprise
- Special/themed: Goalpost Delight, Trophy Cake – Champions Edition, Birthday Blast Berry Bomb, Opera Cake

To tailor things perfectly, could you share a few details?
- Occasion and preferred serving size (roughly how many slices or guests)?
- Flavor direction (chocolate, fruity, creamy, nutty, etc.) and any must-have ingredients or avoidances (nuts, dairy, gluten, vegan)?
- Do you want a themed cake (sports, birthday, elegant) or a classic design?
- Delivery date and 

## 5. Customer Preference Profile

Build long-term customer profiles.

In [12]:
class CustomerProfile:
    """Long-term customer preference tracking"""

    def __init__(self, customer_id, name=None):
        self.customer_id = customer_id
        self.name = name
        self.preferences = {
            'favorite_flavors': [],
            'dietary_restrictions': [],
            'occasions': [],
            'budget_range': None
        }
        self.order_history = []
        self.conversation_summary = ""

    def learn_from_conversation(self, conversation_history):
        """Extract preferences from conversation"""
        # Use LLM to analyze conversation
        analysis_prompt = f"""
        Analyze this conversation and extract customer preferences:

        {conversation_history}

        Extract:
        1. Favorite flavors/types
        2. Dietary restrictions
        3. Common occasions
        4. Budget sensitivity

        Return as JSON.
        """

        response = llm.invoke(analysis_prompt)
        return response.content

    def add_order(self, order_details):
        """Track order history"""
        self.order_history.append(order_details)

    def get_personalized_recommendations(self):
        """Get recommendations based on profile"""
        if not self.preferences['favorite_flavors']:
            return "Need more interaction to provide personalized recommendations"

        rec_prompt = f"""
        Customer Profile:
        - Favorite flavors: {self.preferences['favorite_flavors']}
        - Restrictions: {self.preferences['dietary_restrictions']}
        - Past orders: {len(self.order_history)}

        Available products:
        {cakes_df[['Name', 'Category']].to_string()}

        Recommend 2 products this customer would love and explain why.
        """

        response = llm.invoke(rec_prompt)
        return response.content

# Test customer profile
profile = CustomerProfile(customer_id="C001", name="Emma")
profile.preferences['favorite_flavors'] = ['chocolate', 'dark chocolate']
profile.preferences['dietary_restrictions'] = ['vegan']

print("🎯 Personalized Recommendations for Emma:\n")
print(profile.get_personalized_recommendations())

🎯 Personalized Recommendations for Emma:

Based on the profile (vegan restriction and love for chocolate/dark chocolate), here are two chocolate-forward options from the catalog that would likely appeal most. Note: please confirm the vegan status with the bakery or ingredient list, as the data here doesn’t specify vegan suitability.

1) ChocoCaramel Birthday Surprise (Category G)
- Why it fits: Strong chocolate flavor with caramel, matching the customer’s chocolate/dark chocolate preference. The name suggests a rich, indulgent cake that’s festive and crowd-pleasing for celebrations.
- Vegan note: A vegan version may be possible with plant-based frosting and dairy-free fillings. I can check availability or request vegan substitutions if you’d like.

2) Chocolate Truffle Cake (Category R)
- Why it fits: Explicitly chocolate-focused, offering a rich, dark-chocolate experience that aligns with the customer’s top flavors.
- Vegan note: Often adaptable to vegan with dairy-free ganache/dairy-

## 6. Summary Memory for Long Conversations

Use summary memory for complex customer journeys.

In [13]:
# Summary memory for long interactions
summary_memory = ConversationSummaryMemory(llm=llm)

summary_conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=False
)

# Simulate a long planning conversation
planning_conversation = [
    "I'm planning my wedding and need cakes for 100 guests",
    "The wedding theme is rustic elegance",
    "We have some guests who are gluten-free",
    "Budget is around $500 for all desserts",
    "The wedding is in 3 months",
    "Can we schedule a tasting session?",
]

print("💒 Wedding Planning Conversation\n")
for msg in planning_conversation:
    response = summary_conversation.predict(input=msg)
    print(f"👰 Customer: {msg}")
    print(f"🍰 BakeryAI: {response[:100]}...\n")

# Check the summary
print("\n📝 Conversation Summary:")
print(summary_memory.load_memory_variables({}))

💒 Wedding Planning Conversation



/tmp/ipython-input-2232192512.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=llm)


👰 Customer: I'm planning my wedding and need cakes for 100 guests
🍰 BakeryAI: That sounds fun! For 100 guests, you’ve got some easy, reliable options. Here are common approaches ...

👰 Customer: The wedding theme is rustic elegance
🍰 BakeryAI: Love the rustic elegance direction. Here are three tailored cake options for about 100 guests, each ...

👰 Customer: We have some guests who are gluten-free
🍰 BakeryAI: Great news—gluten-free guests can be accommodated with any of the options. Here are practical ways t...

👰 Customer: Budget is around $500 for all desserts
🍰 BakeryAI: Great constraint. With about $500 for all desserts and ~100 guests, Option C (one or two sheet cakes...

👰 Customer: The wedding is in 3 months
🍰 BakeryAI: Fantastic — with 3 months to go, Option C is well-timed to nail the plan, stay within budget, and en...

👰 Customer: Can we schedule a tasting session?
🍰 BakeryAI: Yes—let’s schedule a tasting. Here are the options and what I’ll need from you to lock it in:

Opti

## 7. Complete BakeryAI System Integration

In [14]:
class CompleteBakeryAI:
    """
    🎉 COMPLETE BAKERYAI v1.0

    Features:
    - Conversation memory
    - Order tracking
    - Customer profiles
    - Semantic search
    - Personalized recommendations
    """

    def __init__(self, product_catalog):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
        self.embeddings = OpenAIEmbeddings()
        self.catalog = product_catalog
        self.active_sessions = {}
        self.customer_profiles = {}

    def start_conversation(self, customer_id, customer_name=None):
        """Initialize or resume customer conversation"""
        if customer_id not in self.active_sessions:
            self.active_sessions[customer_id] = BakeryOrderSession(customer_name)

        if customer_id not in self.customer_profiles:
            self.customer_profiles[customer_id] = CustomerProfile(customer_id, customer_name)

        return f"Welcome{' back' if customer_id in self.customer_profiles else ''}, {customer_name}! 🍰"

    def process_message(self, customer_id, message):
        """Process customer message with full context"""
        session = self.active_sessions.get(customer_id)
        profile = self.customer_profiles.get(customer_id)

        if not session:
            return "Please start a session first"

        # Build comprehensive context
        context = f"""
        You are BakeryAI.

        Current Session:
        {session.get_cart_summary()}
        Preferences: {session.preferences}

        Customer Profile (from past interactions):
        {profile.preferences if profile else 'New customer'}

        Available Products:
        {self.catalog[['Name', 'Description', 'Energy_kcal']].to_string()}

        Conversation History:
        {session.memory.load_memory_variables({})}

        Customer Message: {message}

        Provide a helpful, contextual response.
        """

        response = self.llm.invoke(context)

        # Save to memory
        session.memory.save_context(
            {"input": message},
            {"output": response.content}
        )

        return response.content

    def finalize_order(self, customer_id):
        """Complete and save the order"""
        session = self.active_sessions.get(customer_id)
        profile = self.customer_profiles.get(customer_id)

        if session and session.cart:
            order = session.get_full_context()
            profile.add_order(order)

            # Generate confirmation
            confirmation = f"""
            ✅ Order Confirmed!

            {session.get_cart_summary()}

            Delivery: {session.delivery_info}

            We'll send you updates via email. Thank you for choosing BakeryAI! 🎉
            """

            return confirmation

        return "No items in cart to finalize."

# Initialize Complete BakeryAI
complete_bakery = CompleteBakeryAI(cakes_df)

# Test complete system
customer_id = "VIP_001"
print(complete_bakery.start_conversation(customer_id, "Isabella"))
print()

test_conversation = [
    "Hi! I need a cake for my mother's 60th birthday",
    "She's not a fan of too much chocolate",
    "The vanilla one sounds lovely! Can I get that?",
    "Yes, please add it. Deliver on Saturday afternoon"
]

for msg in test_conversation:
    print(f"💬 Isabella: {msg}")
    response = complete_bakery.process_message(customer_id, msg)
    print(f"🍰 BakeryAI: {response}\n")

# Finalize order
print("\n" + "=" * 70)
print(complete_bakery.finalize_order(customer_id))

Welcome back, Isabella! 🍰

💬 Isabella: Hi! I need a cake for my mother's 60th birthday
🍰 BakeryAI: Hello! Happy birthday to your mother! We have a variety of cakes that would be perfect for her 60th birthday celebration. Some options include the Trophy Cake – Champions Edition, Birthday Blast Berry Bomb, Hazelnut Toffee Crown, or even the Rainbow Cake for a fun and colorful choice. Do you have any specific flavor preferences or dietary restrictions we should take into consideration? Let me know how I can assist you further!

💬 Isabella: She's not a fan of too much chocolate
🍰 BakeryAI: I understand that your mother is not a fan of too much chocolate. Based on that preference, I would recommend considering the Torta della Nonna Amore, Festiva della Sicilia, or the Lemon Drizzle Cake. These options offer a delicious variety of flavors without an overwhelming amount of chocolate. Let me know if any of these options sound appealing to you, or if you have any other preferences I can take in

## 🎯 Exercise 7: Build a Returning Customer Experience

**Task**: Enhance the system to recognize returning customers:
1. Load previous order history
2. Greet them with personalized message
3. Suggest reordering favorites
4. Offer loyalty rewards

In [15]:
class ReturningCustomerHandler:
    def __init__(self, customer_profile):
        self.profile = customer_profile

    def generate_welcome_back_message(self):
        """Create personalized welcome for returning customer"""
        # TODO: Implement personalized welcome
        pass

    def suggest_reorder(self):
        """Suggest reordering previous favorites"""
        # TODO: Analyze order history and suggest
        pass

    def calculate_loyalty_reward(self):
        """Calculate loyalty points and rewards"""
        # TODO: Calculate based on order history
        pass

# Test with mock profile
# mock_profile = CustomerProfile("C100", "David")
# mock_profile.order_history = [...] # Add mock orders
# handler = ReturningCustomerHandler(mock_profile)
# print(handler.generate_welcome_back_message())

## 🎯 Exercise 8: Conversation Analytics

**Task**: Build analytics to understand customer interactions:
1. Average conversation length
2. Most common questions
3. Conversion rate (chat → order)
4. Customer satisfaction indicators

In [16]:
class ConversationAnalytics:
    def __init__(self, all_sessions):
        self.sessions = all_sessions

    def analyze_conversation_patterns(self):
        """Analyze conversation patterns across all customers"""
        # TODO: Extract patterns from conversations
        pass

    def identify_common_questions(self):
        """Find most frequently asked questions"""
        # TODO: Use embeddings to cluster similar questions
        pass

    def calculate_conversion_metrics(self):
        """Calculate chat-to-order conversion rate"""
        # TODO: Analyze which conversations led to orders
        pass

# analytics = ConversationAnalytics(customer_sessions)
# print(analytics.analyze_conversation_patterns())

## 🎉 Congratulations! BakeryAI v1.0 is Complete!

### ✅ Session 1 Complete - What We Built:

#### Session 1.1: Foundation
- ✅ Environment setup and data loading
- ✅ Basic product information chatbot
- ✅ Structured data integration

#### Session 1.2: Intelligence
- ✅ Product embeddings and semantic search
- ✅ Smart product recommendations
- ✅ Multi-provider LLM support

#### Session 1.3: Professionalism
- ✅ Reusable prompt templates
- ✅ Structured outputs (JSON)
- ✅ Multi-language support
- ✅ Email automation

#### Session 1.4: Memory (Current)
- ✅ Conversation memory
- ✅ Session-based order tracking
- ✅ Customer profiles
- ✅ Multi-customer management
- ✅ Complete integrated system

### 🚀 BakeryAI v1.0 Capabilities:

✨ **Natural Conversations**: Multi-turn dialogues with context  
✨ **Smart Search**: Semantic product discovery  
✨ **Personalization**: Remembers preferences  
✨ **Order Management**: Cart tracking and completion  
✨ **Professional Communication**: Consistent brand voice  
✨ **Multi-Customer**: Handle many customers simultaneously  
✨ **Profile Building**: Long-term customer understanding  

### 🔮 What's Next in Future Sessions:

#### Session 2: Agents & Tools
- Add tool-using capabilities
- Calendar integration for delivery scheduling
- Payment processing
- Inventory checking

#### Session 3: RAG (Retrieval Augmented Generation)
- Load company policies and handbooks
- Vector database integration
- Answer policy questions
- Handle complex inquiries

#### Session 4: Deployment
- REST API with LangServe
- Observability with LangSmith
- Production optimizations
- Scaling strategies

### 📊 BakeryAI v1.0 Progress: 100%

```
[████████████████████] 100% ✨
```

### 🎊 You now have a functional AI customer service agent!

The foundation is solid. In the coming sessions, we'll add:
- External tools and integrations
- Knowledge base access
- Production deployment
- Advanced workflows

**Great work completing Day 1! See you in Session 2! 🚀**